In [3]:
import tensorflow as tf
import tensorflowvisu
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
import math
print("Tensorflow version " + tf.__version__)
tf.set_random_seed(0)

mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)


Tensorflow version 1.3.0
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [20]:
pkeep = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 28, 28, 1])

W1 = tf.Variable(tf.truncated_normal([6, 6, 1, 6] ,stddev=0.1))
B1 = tf.Variable(tf.ones([6])/10)

W2 = tf.Variable(tf.truncated_normal([5, 5, 6, 12] ,stddev=0.1))
B2 = tf.Variable(tf.ones([12])/10)

W3 = tf.Variable(tf.truncated_normal([4, 4, 12, 24] ,stddev=0.1))
B3 = tf.Variable(tf.ones([24])/10)

W4 = tf.Variable(tf.truncated_normal([7*7*24, 200] ,stddev=0.1))
B4 = tf.Variable(tf.ones([200])/10)

W5 = tf.Variable(tf.truncated_normal([200, 10] ,stddev=0.1))
B5 = tf.Variable(tf.ones([10])/10)

stride=1
Y1cnv = tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME')
Y1 = tf.nn.relu(Y1cnv + B1)

stride = 2
Y2cnv = tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME')
Y2 = tf.nn.relu(Y2cnv + B2)

Y3cnv = tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME')
Y3 = tf.nn.relu(Y3cnv + B3)

Y3rsh = tf.reshape(Y3, [-1,7*7*24])

Y4 = tf.nn.relu(tf.matmul(Y3rsh, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)

Ylogits = tf.matmul(Y4d, W5) + B5
Y = tf.nn.softmax(Ylogits)

Y_ = tf.placeholder(tf.float32, [None, 10])

lr = tf.placeholder(tf.float32)

In [21]:
#cross_entropy = -tf.reduce_mean(Y_ * tf.log(Y)) * 1000.0  # normalized for batches of 100 images,
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100


correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.train.AdamOptimizer(lr)
train_step= optimizer.minimize(cross_entropy)

In [22]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

def training_step(i):
    batch_X, batch_Y = mnist.train.next_batch(100)
    
    max_learning_rate = 0.003
    min_learning_rate = 0.0001
    decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    train_data = {X: batch_X, Y_: batch_Y, lr: learning_rate, pkeep: 0.5}
        
    if i%100 == 0:
        a = sess.run(accuracy, feed_dict={X: mnist.test.images, Y_: mnist.test.labels, pkeep: 1})
        print(str(i) + ' test accuracy: ' + str(a))
        
        
    sess.run(train_step, feed_dict=train_data)

In [23]:
for i in range(10000+1): training_step(i)

0 test accuracy: 0.0989
100 test accuracy: 0.9476
200 test accuracy: 0.9608
300 test accuracy: 0.9725
400 test accuracy: 0.9769
500 test accuracy: 0.9788
600 test accuracy: 0.9818
700 test accuracy: 0.9847
800 test accuracy: 0.9817
900 test accuracy: 0.9854
1000 test accuracy: 0.9881
1100 test accuracy: 0.988
1200 test accuracy: 0.9863
1300 test accuracy: 0.9878
1400 test accuracy: 0.9866
1500 test accuracy: 0.9881
1600 test accuracy: 0.9889
1700 test accuracy: 0.9884
1800 test accuracy: 0.9847
1900 test accuracy: 0.9882
2000 test accuracy: 0.9897
2100 test accuracy: 0.9893
2200 test accuracy: 0.9892
2300 test accuracy: 0.9897
2400 test accuracy: 0.991
2500 test accuracy: 0.9906
2600 test accuracy: 0.9892
2700 test accuracy: 0.9897
2800 test accuracy: 0.9894
2900 test accuracy: 0.9905
3000 test accuracy: 0.9908
3100 test accuracy: 0.9905
3200 test accuracy: 0.9913
3300 test accuracy: 0.9906
3400 test accuracy: 0.9912
3500 test accuracy: 0.9906
3600 test accuracy: 0.9907
3700 test accur